# Automate scenario runs

The process involves:
1. running a scenario-specific spinup, initialized with `spin3.out.press.pfb` using `scen_spin.tcl` (this will simulate 5 years of 2010 meteorological forcing
2. Running the variable dz subsurface post processing script (python with fortran module, which needs to be installed using: `f2py -c -m postproc scale_pfb.f pfb_read.f writepfb.f` and has dependencies: `scale_pfb.f pfb_read.f writepfb.f`) Need to confirm that subsurface storage is changing by less than 1% between years 4 and 5.
3. Use the last pfb file `slopes_only.out.press.43805.pfb` as the input `spin4.out.press.pfb` (save this in each scen's directory) to the one year simulation
4. Remove all spinup run files to make space
5. Run the 1-year simulation using `scen_run.tcl`
6. Run the full water balance script using `postproc` f2py module. Save the csv dataframe in outputs directory.
7. Remove all run files.

In [1]:
import subprocess
import glob
import os
import shutil
import time

from postproc import scale_pfb
import pandas as pd
from io import StringIO
import itertools

%matplotlib inline

In [29]:
homedir = os.path.expanduser("~")
homedir

'/home/tclim'

In [9]:
scendir_base = '/home/tclim/Dropbox/projects/pf_simple/00preproc/01subsurf_scen'

# Copy scenario subsurface inputs into run directory

In [4]:
scen = 'scen000'

In [12]:
[print (f) for f in glob.glob('%s/%s/*' %(scendir_base, scen))]


/home/tclim/Dropbox/projects/pf_simple/00preproc/01subsurf_scen/scen000/mann_scen.pfb
/home/tclim/Dropbox/projects/pf_simple/00preproc/01subsurf_scen/scen000/perm_scen.pfb
/home/tclim/Dropbox/projects/pf_simple/00preproc/01subsurf_scen/scen000/ind_scen.pfb


[None, None, None]

In [10]:
'%s/%s/*' %(scendir_base, scen)

'/home/tclim/Dropbox/projects/pf_simple/00preproc/01subsurf_scen/scen000/*'

In [15]:
shutil.copy2('%s/%s/mann_scen.pfb' %(scendir_base, scen), '.')
shutil.copy2('%s/%s/ind_scen.pfb' %(scendir_base, scen), '.')
shutil.copy2('%s/%s/perm_scen.pfb' %(scendir_base, scen), '.')

'./perm_scen.pfb'

# Run Scenario-specific spinup

In [22]:
start = time.time()
bashCommand = "tclsh scen_spin.tcl"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Time Elapsed:  0.6267449855804443 s


In [23]:
print(output.decode("utf-8").format())


ParFlow run complete
Using process grid (1,1,1)
Using process grid (1,1,1)
Using process grid (1,1,1)
Using process grid (1,1,1)



# Calculate vdz_subsurf storage

Time Elapsed:  0.024671077728271484 s
